# Statistical Analysis of Permian Basin Equipment Maintenance at Chevron


## Issue
Performing a solid statistical analysis with the given dataset (workorderdata.csv) presented considerable issues since the beginning. The data is incomplete and full of NaN, and dropping all these entries ended up in a huge percentage of data loss of about 60%. In any other situation, this would be acceptable because even after cleaning the data we would have around 500k entries, but considering that the dataset represents a time series of maintenance jobs and that we are trying to predict when the equipment will fail again dropping the entries would end up in skew and unprecise results.  

## Solution
To work around this problem I decided to only consider the columns without any NaN in them, assuming that they are 100% complete because they represent the most important data in the logging of every job.
Said columns are: 
"WorkOrder", "SupervisorRole", "TradeGroup",  "CreatedDate",                                "IsAffectingProduction",                                 "AffectedProduction",                            "GrossProductionLoss", "Duration", "Safety", "WOType", "Reopened", "StatusDescription", "EquipmentCode",
"StatusCode" and                       "EquipmentType".

## EDA


In [1]:
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize
from sklearn.preprocessing import LabelEncoder

In [2]:
worker_order_df = pd.read_csv('workorderdata/workorderdata.csv')

/var/folders/cl/s5bcg3hn4tv5q_rwxx9sgg5r0000gn/T/ipykernel_99350/99475405.py:1: DtypeWarning: Columns (43,46,48,49,52,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  worker_order_df = pd.read_csv('workorderdata/workorderdata.csv')


Before dropping all the unnecessary columns, a simple analysis was made to determine how the data was going to be split according to the job type preventive or corrective. 

Solution: Split the dataset into PPM and (JOB + XA + AA + STAT + IN + MRC + XL + PL)

In [3]:
test = worker_order_df[["WOType", "JobType"]]

ppm_df = test.query('WOType == "PPM"').dropna()
print("PPM", ppm_df.shape)

job_df = test.query('WOType == "JOB"').dropna()
print("JOB", job_df.shape)

xa_df = test.query('WOType == "XA"').dropna()
print("XA", xa_df.shape)

aa_df = test.query('WOType == "AA"').dropna()
print("AA", aa_df.shape)

stat_df = test.query('WOType == "STAT"').dropna()
print("STAT", stat_df.shape)

in_df = test.query('WOType == "IN"').dropna()
print("IN", in_df.shape)

mrc_df = test.query('WOType == "MRC"').dropna()
print("MRC", mrc_df.shape)

xl_df = test.query('WOType == "XL"').dropna()
print("XL", xl_df.shape)

pl_df = test.query('WOType == "PL"').dropna()
print("PL", pl_df.shape)

PPM (651628, 2)
JOB (580288, 2)
XA (0, 2)
AA (0, 2)
STAT (0, 2)
IN (0, 2)
MRC (0, 2)
XL (0, 2)
PL (0, 2)


Dropping Columns. "StatusDescription" was dropped to because is the long form of "StatusCode"

In [4]:
worker_order_df = worker_order_df[["WorkOrder",
                                   "SupervisorRole",
                                   "TradeGroup",
                                   "CreatedDate",
                                   "IsAffectingProduction",
                                   "AffectedProduction",
                                   "GrossProductionLoss",
                                   "Duration",
                                   "Safety",
                                   "WOType",
                                   "Reopened",
                                   # "StatusDescription", Deleted because is the long form of StatusCode
                                   "EquipmentCode",
                                   "StatusCode",
                                   "EquipmentType"]]
worker_order_df.head()

,WorkOrder,SupervisorRole,TradeGroup,CreatedDate,IsAffectingProduction,AffectedProduction,GrossProductionLoss,Duration,Safety,WOType,Reopened,EquipmentCode,StatusCode,EquipmentType
0,DS10001182,Surface Maintenance Supervisor,Mechanical,2008-09-30,No,0,0,30,N,JOB,N,EO9659,C,System
1,DS10001195,Surface Maintenance Supervisor,Mechanical,2008-09-30,No,0,0,7,N,JOB,N,79A0052188,C,Asset
2,DS10001202,Production Supervisor,Mechanical,2008-09-30,No,0,0,1,N,JOB,N,CY3417WH,C,Position
3,DS10001205,Surface Maintenance Supervisor,Mechanical,2008-09-30,Yes,8,0,1,N,JOB,N,22B0361288,C,Asset
4,DS10001226,Surface Maintenance Supervisor,Electric,2008-09-30,No,0,0,7,N,JOB,N,BCT49CVUBP4V2,C,Position


Analysis of the categorical variables

In [5]:
columns = worker_order_df.columns
id_vars = []
for column in columns:
    uniques = worker_order_df[column].unique()
    if len(uniques) < 100:
        print(worker_order_df.groupby([column]).size().sort_values(ascending=False))
        print()
    else:
        id_vars.append(column)
print("Non categorical variables", id_vars, sep="\n")


SupervisorRole
Surface Maintenance Supervisor    1076675
Production Supervisor              243015
IE Supervisor                       61211
Surface Maintenance Advisor         49018
dtype: int64

TradeGroup
Mechanical    1154732
Electric       178429
Admin           78285
Other           18473
dtype: int64

IsAffectingProduction
No     1350135
Yes      79784
dtype: int64

Safety
N    1424354
Y       5565
dtype: int64

WOType
PPM     651628
JOB     580288
XA       70133
AA       58136
STAT     32318
IN       18398
MRC       9716
XL        9298
PL           4
dtype: int64

Reopened
N    1418644
Y      11275
dtype: int64

StatusCode
C       1285288
A         62753
REJ       26693
CANC      21344
R         15215
D          9404
SCH        4527
SJOB       1847
SD         1030
WM          558
B           410
HC          403
Q           165
HE          123
MOC          79
EC           56
HR           11
REO           6
RFMI          6
MIN           1
dtype: int64

EquipmentType
Position    5

Data split, corrective and preventive jobs

In [6]:
job_df = worker_order_df.query('WOType != "PPM"')
ppm_df = worker_order_df.query('WOType == "PPM"')
job_df.shape

(778291, 14)

Analysing what equipments were the ones with most failures

In [7]:
job_df.groupby(["EquipmentCode"]).size().sort_values(ascending=False).head(30)

EquipmentCode
C35              5465
V60              2943
MWTP             2286
0081             2080
J48              1920
SKINNER-RIDGE    1830
CO2GEN           1747
NMO-L            1440
59836AVF         1384
WEWIS            1376
V60MBS           1368
VAN              1242
C35INJ           1196
BUCKEYE          1023
BI2               920
VEALMOOR          861
C35SWP            852
BECKVILLE         801
BCT49CVUB         777
KR                766
C35O              755
BP8               752
HDGP              747
C35CS04           735
BPJ6CRFCC         714
NGL1              713
DEADWOOD          653
CO2T1             630
C35CS05           625
C35CS47           609
dtype: int64

Discrovered that many eqiupments had many jobs at the same day

In [8]:
job_df.groupby(["EquipmentCode", "CreatedDate"]).size().sort_values(ascending=False).head(30)

EquipmentCode  CreatedDate
V60            2006-05-27     362
BCT49CVUB      2006-05-27     330
BU50           2006-05-27     322
V60BISYS       2006-05-27     289
0081           2006-05-27     266
H33BF2CS       2006-05-27     252
C35            2006-05-27     194
BCT49VGSAUB    2006-05-27     191
59836AVF       2009-08-17     186
H33            2006-05-27     178
CO2T1          2006-05-27     169
NMO-L          2006-05-27     162
UCT49VGWUB     2006-05-27     153
V60MBS         2006-05-27     149
WCBATT2        2006-05-27     135
C35SWP         2006-05-27     132
BCT49CVUIJ     2006-05-27     121
H33BF18CS      2006-05-27     114
UCT7NVAWUB     2006-05-27     114
UCT49WVUB      2006-05-27     110
LK8SU940BW     2006-05-27     105
V60205CSB      2006-05-27      97
H33BF16CS      2006-05-27      97
C35O           2006-05-27      96
MWTP           2011-10-26      95
UCT49VDB       2006-05-27      91
V60            2016-10-07      84
UCT49ARB       2006-05-27      84
NMO            2006-0

Grouped all the entries for the same equipement in the same day and keeping the one with the highest duration

In [9]:
same_day_jobs_df = job_df.loc[job_df.groupby(["EquipmentCode", "CreatedDate"])['Duration'].idxmax()].reset_index(drop=True)

Reduced the data set from entried 778291 to 604397 entries

In [10]:
same_day_jobs_df.shape

(604397, 14)

Second analysis of the equipments with the most jobs

In [11]:
# Equipments that cause 80% of the problems
equipment_jobs_ser = same_day_jobs_df.groupby(["EquipmentCode"]).size().sort_values(ascending=False)
print(equipment_jobs_ser)
sum_equipment_jobs_ser = equipment_jobs_ser.cumsum()
job_80_ser = sum_equipment_jobs_ser[sum_equipment_jobs_ser< same_day_jobs_df.shape[0] * 0.8]
print(job_80_ser, "\n")
print("Deleted: ",equipment_jobs_ser.shape[0] - job_80_ser.shape[0])

EquipmentCode
V60                        1160
C35                        1152
MWTP                        948
0081                        941
SKINNER-RIDGE               910
                           ... 
EO1656                        1
EO1655                        1
EO1466POC                     1
EO1466FL                      1
ZOROSAKWP-UTIL-ZZZ0027B       1
Length: 177000, dtype: int64
EquipmentCode
V60                1160
C35                2312
MWTP               3260
0081               4201
SKINNER-RIDGE      5111
                  ...  
77A0056768       483509
66H0070301       483511
22B5308413       483513
66H0070302       483515
66H0070303       483517
Length: 73274, dtype: int64 

Deleted:  103726


To reduce the data we are only studying the equipments that cause 80% of the jobs. Reducing the data set from 604397 to 483517. Also added the column "FailureCount"

In [12]:
job_80_df = same_day_jobs_df[same_day_jobs_df['EquipmentCode'].isin(job_80_ser.keys())]
job_80_df = job_80_df.set_index('EquipmentCode').assign(FailureCount=job_80_df['EquipmentCode'].value_counts()).reset_index()
job_80_df = job_80_df.sort_values(by=["FailureCount", "CreatedDate"], ascending=[False, True])
print(job_80_df.shape)
job_80_df.head()

(483517, 15)


,EquipmentCode,WorkOrder,SupervisorRole,TradeGroup,CreatedDate,IsAffectingProduction,AffectedProduction,GrossProductionLoss,Duration,Safety,WOType,Reopened,StatusCode,EquipmentType,FailureCount
464851,V60,DS3286979,Surface Maintenance Supervisor,Admin,2006-05-27,No,0,0,45,N,JOB,N,C,System,1160
464852,V60,DS8001035,Surface Maintenance Supervisor,Mechanical,2006-06-09,No,0,593243,10,N,JOB,N,C,System,1160
464853,V60,DS8013075,Surface Maintenance Supervisor,Mechanical,2006-06-13,No,0,0,45,N,JOB,N,C,System,1160
464854,V60,DS8014023,Surface Maintenance Supervisor,Mechanical,2006-06-14,No,0,0,1,N,JOB,N,C,System,1160
464855,V60,DS8019382,Surface Maintenance Supervisor,Mechanical,2006-06-22,No,0,0,1,N,JOB,N,C,System,1160


Second analysis of the categorical variables

In [13]:
columns = job_80_df.columns
id_vars = []
for column in columns:
    uniques = job_80_df[column].unique()
    if len(uniques) < 100:
        print(job_80_df.groupby([column]).size().sort_values(ascending=False))
        print()
    else:
        id_vars.append(column)
print("Non categorical variables", id_vars, sep="\n")

SupervisorRole
Surface Maintenance Supervisor    388005
Production Supervisor              49756
IE Supervisor                      38599
Surface Maintenance Advisor         7157
dtype: int64

TradeGroup
Mechanical    375261
Electric       82085
Admin          23107
Other           3064
dtype: int64

IsAffectingProduction
No     420837
Yes     62680
dtype: int64

Safety
N    479010
Y      4507
dtype: int64

WOType
JOB     411966
XA       40193
XL        8042
STAT      7969
MRC       7178
IN        6132
AA        2035
PL           2
dtype: int64

Reopened
N    477655
Y      5862
dtype: int64

StatusCode
C       457949
REJ      17464
D         3630
R         1577
SCH       1023
SJOB       886
SD         283
WM         251
HC         214
Q          116
HE          59
MOC         48
EC           9
HR           4
REO          2
RFMI         2
dtype: int64

EquipmentType
System      218289
Position    157710
Asset       104346
Location      3172
dtype: int64

Non categorical variables
['Equi

Conversion of string columns to numeric columns

In [14]:
columns = job_80_df.columns
seeds_matrix = []
for column in columns:
    uniques = job_80_df[column].unique()
    if len(uniques) == 2:
        keys = job_80_df.groupby([column]).size().sort_values(ascending=False).keys()
        job_80_df[column].replace(keys, [0,1], inplace=True)
        i = -1
        print(column)
        for key in keys:
            i += 1
            print(key, i, sep=" Converted to: ")
        print('\n')
    elif 2 < len(uniques) < 100:
        keys = job_80_df.groupby([column]).size().sort_values(ascending=False).keys()
        seed = list(range(keys.shape[0]))
        np.random.shuffle(seed)
        job_80_df[column].replace(keys, seed, inplace=True)
        seeds_matrix.append(seed)
        for i in range(keys.shape[0]):
            print(keys[i], seed[i], sep=" Converted to: ")
        print('\n')

Surface Maintenance Supervisor Converted to: 2
Production Supervisor Converted to: 0
IE Supervisor Converted to: 3
Surface Maintenance Advisor Converted to: 1


Mechanical Converted to: 0
Electric Converted to: 2
Admin Converted to: 1
Other Converted to: 3


IsAffectingProduction
No Converted to: 0
Yes Converted to: 1


Safety
N Converted to: 0
Y Converted to: 1


JOB Converted to: 1
XA Converted to: 0
XL Converted to: 7
STAT Converted to: 4
MRC Converted to: 6
IN Converted to: 3
AA Converted to: 5
PL Converted to: 2


Reopened
N Converted to: 0
Y Converted to: 1


C Converted to: 7
REJ Converted to: 11
D Converted to: 1
R Converted to: 0
SCH Converted to: 4
SJOB Converted to: 6
SD Converted to: 12
WM Converted to: 10
HC Converted to: 15
Q Converted to: 13
HE Converted to: 8
MOC Converted to: 14
EC Converted to: 3
HR Converted to: 5
REO Converted to: 2
RFMI Converted to: 9


System Converted to: 1
Position Converted to: 0
Asset Converted to: 3
Location Converted to: 2




In [15]:
job_80_df.head()

,EquipmentCode,WorkOrder,SupervisorRole,TradeGroup,CreatedDate,IsAffectingProduction,AffectedProduction,GrossProductionLoss,Duration,Safety,WOType,Reopened,StatusCode,EquipmentType,FailureCount
464851,V60,DS3286979,2,1,2006-05-27,0,0,0,45,0,1,0,7,1,1160
464852,V60,DS8001035,2,0,2006-06-09,0,0,593243,10,0,1,0,7,1,1160
464853,V60,DS8013075,2,0,2006-06-13,0,0,0,45,0,1,0,7,1,1160
464854,V60,DS8014023,2,0,2006-06-14,0,0,0,1,0,1,0,7,1,1160
464855,V60,DS8019382,2,0,2006-06-22,0,0,0,1,0,1,0,7,1,1160


To determine the time since an equipment had a failure the column "CreatedDate" was converted to a date type

In [16]:
job_80_df['CreatedDate'] = job_80_df['CreatedDate'].apply(pd.to_datetime)
job_80_df = job_80_df.reset_index(drop=True)

Generation of column "DaysFromLastFailure", the first entrie of every equipment is NaT since there are not values to compare. Still need to decide what to do with that

In [17]:
job_80_df = job_80_df.assign(DaysFromLastFailure = job_80_df.groupby('EquipmentCode')['CreatedDate'].diff())

In [18]:
job_80_df.head()

,EquipmentCode,WorkOrder,SupervisorRole,TradeGroup,CreatedDate,IsAffectingProduction,AffectedProduction,GrossProductionLoss,Duration,Safety,WOType,Reopened,StatusCode,EquipmentType,FailureCount,DaysFromLastFailure
0,V60,DS3286979,2,1,2006-05-27,0,0,0,45,0,1,0,7,1,1160,NaT
1,V60,DS8001035,2,0,2006-06-09,0,0,593243,10,0,1,0,7,1,1160,13 days
2,V60,DS8013075,2,0,2006-06-13,0,0,0,45,0,1,0,7,1,1160,4 days
3,V60,DS8014023,2,0,2006-06-14,0,0,0,1,0,1,0,7,1,1160,1 days
4,V60,DS8019382,2,0,2006-06-22,0,0,0,1,0,1,0,7,1,1160,8 days


Sort the dataset by "EquipmentCode" and "CreatedDate"


In [19]:
job_80_df_sorted = job_80_df.sort_values(['EquipmentCode','CreatedDate'],ascending=[False, True])
print(job_80_df.shape)
job_80_df_sorted = job_80_df_sorted.reset_index(drop=True)
job_80_df_sorted.head()
# job_80_df_sorted[job_80_df_sorted["EquipmentCode"] == "LH-PLT-3C-FSG-P11A" ]
# pd.set_option('display.max_rows', 100)
# job_80_df_sorted.tail(100)

(483517, 16)


,EquipmentCode,WorkOrder,SupervisorRole,TradeGroup,CreatedDate,IsAffectingProduction,AffectedProduction,GrossProductionLoss,Duration,Safety,WOType,Reopened,StatusCode,EquipmentType,FailureCount,DaysFromLastFailure
0,ZOROSAKWP,DS17161038,2,3,2016-10-13,0,0,0,1,0,1,0,7,0,5,NaT
1,ZOROSAKWP,DS17282768,0,0,2017-01-11,0,0,0,1,0,1,0,7,0,5,90 days
2,ZOROSAKWP,DS17668787,0,0,2017-09-21,0,0,0,1,0,1,0,7,0,5,253 days
3,ZOROSAKWP,DS17836166,0,0,2018-01-08,1,158,158,1,0,1,0,7,0,5,109 days
4,ZOROSAKWP,DS17974621,0,0,2018-04-05,0,0,0,1,0,1,0,1,0,5,87 days


# Second Sprint

In [20]:
job_df_clean = job_80_df_sorted

In [21]:
# Transform date to day of the year to get seasonality
job_df_clean['DayOfYear'] = job_df_clean['CreatedDate'].dt.dayofyear

In [22]:
# Filling NaT with the mean of it's id 
grouped = job_df_clean.groupby('EquipmentCode')['DaysFromLastFailure'].transform(lambda x: np.mean(x.dropna()))
job_df_clean['DaysFromLastFailure'] = job_df_clean['DaysFromLastFailure'].fillna(grouped)

In [23]:
# Encode Equipment Code
le = LabelEncoder()
job_df_clean['equipment_model_id_encoded'] = le.fit_transform(job_df_clean['EquipmentCode'])

In [24]:
# Transform Date to Int
job_df_clean['DaysFromLastFailure_int'] = job_df_clean['DaysFromLastFailure'].dt.days.astype('int32')

In [25]:
# Drop useless columns
job_df_clean = job_df_clean.drop(columns=['WorkOrder', 'CreatedDate', 'EquipmentCode', 'DaysFromLastFailure' ])

In [26]:
job_df_clean.head()

,SupervisorRole,TradeGroup,IsAffectingProduction,AffectedProduction,GrossProductionLoss,Duration,Safety,WOType,Reopened,StatusCode,EquipmentType,FailureCount,DayOfYear,equipment_model_id_encoded,DaysFromLastFailure_int
0,2,3,0,0,0,1,0,1,0,7,0,5,287,73273,134
1,0,0,0,0,0,1,0,1,0,7,0,5,11,73273,90
2,0,0,0,0,0,1,0,1,0,7,0,5,264,73273,253
3,0,0,1,158,158,1,0,1,0,7,0,5,8,73273,109
4,0,0,0,0,0,1,0,1,0,1,0,5,95,73273,87


In [27]:
# Find Outliers
outliers = job_df_clean[['AffectedProduction','GrossProductionLoss','Duration', 'DaysFromLastFailure_int']].quantile([0.25,0.5, 0.9, 0.95, 0.99, 0.995, 0.999]).transpose()
outliers['max'] = job_df_clean[['AffectedProduction','GrossProductionLoss','Duration','DaysFromLastFailure_int' ]].max()
outliers['mean'] = job_df_clean[['AffectedProduction','GrossProductionLoss','Duration', 'DaysFromLastFailure_int']].mean()
outliers['min'] = job_df_clean[['AffectedProduction','GrossProductionLoss','Duration', 'DaysFromLastFailure_int']].min()

outliers

,0.25,0.5,0.9,0.95,0.99,0.995,0.999,max,mean,min
AffectedProduction,0.0,0.0,4.0,14.0,75.0,150.00,940.968,318455302,671.439087,0
GrossProductionLoss,0.0,0.0,0.0,0.0,100.0,300.00,594062.484,595336,1436.399883,0
Duration,1.0,1.0,7.0,7.0,93.0,93.00,259.484,687389,9.072405,1
DaysFromLastFailure_int,24.0,127.0,1171.0,1936.0,3565.0,3916.42,4216.000,5081,410.142711,1


In [28]:
job_df_clean.to_csv("cleanworkorderoutliers.csv", index=False)
print("Done")

Done


In [29]:
# Kill outliers with winsorize
winsorized_data = winsorize(job_df_clean['AffectedProduction'], limits=[0, 1-0.999])
job_df_clean['AffectedProduction'] = winsorized_data

winsorized_data = winsorize(job_df_clean['GrossProductionLoss'], limits=[0, 1-0.995])
job_df_clean['GrossProductionLoss'] = winsorized_data

winsorized_data = winsorize(job_df_clean['Duration'], limits=[0, 1-0.999])
job_df_clean['Duration'] = winsorized_data

In [30]:
outliers = job_df_clean[['AffectedProduction','GrossProductionLoss','Duration', 'DaysFromLastFailure_int']].quantile([0.25,0.5, 0.9, 0.95, 0.99, 0.995, 0.999]).transpose()
outliers['max'] = job_df_clean[['AffectedProduction','GrossProductionLoss','Duration','DaysFromLastFailure_int' ]].max()
outliers['mean'] = job_df_clean[['AffectedProduction','GrossProductionLoss','Duration', 'DaysFromLastFailure_int']].mean()
outliers['min'] = job_df_clean[['AffectedProduction','GrossProductionLoss','Duration', 'DaysFromLastFailure_int']].min()

outliers

,0.25,0.5,0.9,0.95,0.99,0.995,0.999,max,mean,min
AffectedProduction,0.0,0.0,4.0,14.0,75.0,150.00,940.968,942,4.484533,0
GrossProductionLoss,0.0,0.0,0.0,0.0,100.0,300.00,300.000,300,3.163808,0
Duration,1.0,1.0,7.0,7.0,93.0,93.00,259.484,260,3.905699,1
DaysFromLastFailure_int,24.0,127.0,1171.0,1936.0,3565.0,3916.42,4216.000,5081,410.142711,1


In [31]:
job_df_clean.to_csv("cleanworkorder.csv", index=False)
print("Done")

Done
